# A simple version of bolometer noise

Photon, Phonon, and Johnson

J. Ruhl, 6/8/2021

In [1]:
# Standard imports
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline  
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [12, 20]

h = 6.6e-34
kb = 1.38e-23

## Photon noise

Let's consider a single-moded bolometer, operating at center frequency $\nu_0$, with bandwidth $\Delta \nu$, and optical efficiency $\eta$, looking at an effective Rayleigh-Jeans temperature $T$.  The power absorbed by the bolometer is

$$ P = \eta k T \Delta \nu .$$

Note that we can convert between power and (Rayleigh-Jeans) temperature units using a factor of $\eta k \Delta \nu$.

The number of photons that hit it (on average) per second is 

$$ n_\gamma = P/(h \nu_0)  .$$

The random arrival of (uncorrelated) photons means that average fluctuates around by 

$$ \sigma_\gamma = \sqrt{n_\gamma} ,$$

so the rms power fluctuations in one-second integrations is

$$ \sigma_P = h \nu_0 \sigma_\gamma = h \nu_0 \sqrt{n_\gamma} .$$  


Note that we are ignoring the correlated photon (Bose) noise in this approximation.


In [42]:
eta = 0.5
nu0 = 150e9
delta_nu = 0.25*nu0 #25e9   # this is the full bandwidth
T = 10

P = eta*kb*T*delta_nu          # absorbed power
n_gamma = P/(h*nu0)            # number of photons absorbed per second, on average
sigma_gamma = np.sqrt(n_gamma) # rms fluctuations in photons absorbed/second
sigma_P = h*nu0*sigma_gamma    # rms fluctuations in power absorbed/second
sigma_T = sigma_P/(eta*kb*delta_nu)  # effective fluctuations in sky temp (RJ)

print('P =           {0:6.3e}'.format(P))
print('n_gamma =     {0:6.3e}'.format(n_gamma))
print('sigma_gamma = {0:6.3e}'.format(sigma_gamma))
print('sigma_P =     {0:6.3e}'.format(sigma_P))
print('sigma_T =     {0:6.3e}'.format(sigma_T))



P =           2.588e-12
n_gamma =     2.614e+10
sigma_gamma = 1.617e+05
sigma_P =     1.601e-17
sigma_T =     6.186e-05


## Photon noise including the Bose term

The photon noise can be written either in terms of the power, or the occupation number, $n$, not to be confused with $n_\gamma$ above.

The photon NEP is given by

$$ NEP^2_{photon} = \int A(\nu) \left[n(\nu) + n(\nu)^2\right] d\nu ,$$

where $A(\nu) = 2*h^2 \nu^2$.

In [43]:
Tb = T  # Note that we're using Planck brightness temperature here.

nu_low = nu0 - delta_nu/2
nu_high = nu0 + delta_nu/2
nuvec = np.linspace(nu_low,nu_high,1000)

A = 2*(h**2)*(nuvec**2)   # single mode, single polarization

x = h*nuvec/(kb*Tb)

#occupation number
n = eta/(np.exp(x)-1.)    

# photon power on bolometer at each freq
Pnu = h*nuvec*n #power per mode per Hz; times Nmodes*Npol if those are not 1.

#Integrate P to get the total power integrated across band
Ptot= np.trapz(Pnu, nuvec)

#photon noise integral
integrand = A*(n + n**2) 
NEP2_photon = np.trapz(integrand, nuvec)
NEP_photon = np.sqrt(NEP2_photon)
NET_photon = (NEP_photon/np.sqrt(2))/(eta*kb*delta_nu)

#Repeat ignoring Bose term
integrand = A*n
NEP2_photon_poisson = np.trapz(integrand, nuvec)
NEP_photon_poisson = np.sqrt(NEP2_photon_poisson)
NET_photon_poisson = (NEP_photon_poisson/np.sqrt(2))/(eta*kb*delta_nu)

n_bar = np.mean(n)

print('P =           {0:6.3e}'.format(Ptot))
print('Avg n = {0:6.3e}'.format(n_bar))
print('With Bose:')
print('  NEP_photon =  {0:6.3e}  W/sqrt(Hz)'.format(NEP_photon))
print('  NET_photon =  {0:6.3e}  K*sqrt(s)'.format(NET_photon))
print('Without Bose:')
print('  NEP_photon =  {0:6.3e}  W/sqrt(Hz)'.format(NEP_photon_poisson))
print('  NET_photon =  {0:6.3e}  K*sqrt(s)'.format(NET_photon_poisson))


P =           1.770e-12
Avg n = 4.803e-01
With Bose:
  NEP_photon =  2.274e-17  W/sqrt(Hz)
  NET_photon =  6.213e-05  K*sqrt(s)
Without Bose:
  NEP_photon =  1.870e-17  W/sqrt(Hz)
  NET_photon =  5.111e-05  K*sqrt(s)


# Phonon noise

For an isothermal weak thermal link, 

$$ NEP_{phonon} = \sqrt{4 k T^2 G_{dyn}}  .$$

This gets modified because of the temperature gradient across the link, to 

$$ NEP_{phonon} = \sqrt{4 k T^2 G_{dyn} F_{link}}  ,$$

where $F_{link}$ depends on the temperature dependence of the thermal conductance of the link, parameterized by $\beta$ or $n$.  Typically $F_{link}$ is a little smaller than one, but we'll take it to be one from here on.

Note that if

$$ P_{sat} = k ( T_{bolo}^n - T_{bath}^n ) , $$

taking the derivative with w.r.t. T_{bolo} gives

$$ G_{dyn} = \frac{dP_{sat}}{d T_{bolo}} = n k T_{bolo}^{n-1} .$$

If we know $P_{sat}$ and the temperatures, we can solve for $k$ and use that in our formula for $G_{dyn}$.

In [46]:
Tbolo = .16
Tbath = 0.1
nlink = 2.7
Psat = 10e-12

klink = Psat/(Tbolo**nlink - Tbath**nlink)

Gdyn = nlink*klink*Tbolo**(nlink-1)

NEP_phonon = np.sqrt(4*kb*Tbolo**2*Gdyn)

print('Gdyn = {0:6.3e}'.format(Gdyn))
print('NEP_phonon = {0:6.3e}'.format(NEP_phonon))

Gdyn = 2.347e-10
NEP_phonon = 1.821e-17


# Johnson noise

Naiively, 

$$ NEI_{Johnson} = \sqrt{4 k T/R} $$

$$ NEP_{Johnson} = \frac{1}{S_I}\sqrt{4 k T/R} $$

where $S_I$ is the current-to-power responsibility.

However, this noise is suppressed because the loop gain partially cancels it.  That is, in a voltage-biased bolometer, a Johnson noise fluctuation causes the current through the bolometer to change, changing the electrical power dissipation in a way that acts (through electro thermal feedback) to cancel some of the current change.

So, the end result is more like

$$ NEP_{Johnson} = \frac{1}{S_I L_0} \sqrt{4 k T/R} .$$

The current responsivity is 

$$ S_I = \frac{dI}{dP} $$, 

which we can find for a high-loopgain, voltage-biased bolometer by considering that to a close approximation $R =$ constant, so

$$ \Delta P_{elec} = - \Delta P_{opt} ,$$

$$ \Delta (I V_{bolo}) = - \Delta P_{opt} ,$$

$$ V \Delta I = - \Delta P_{opt} ,$$

$$ \frac{\Delta I}{\Delta P_{opt}}  = \frac{dI}{dP} = \frac{-1}{V_{bolo}} ,  $$

$$ S_I = \frac{-1}{V_{bolo}} . $$


In [50]:
Rbolo = 0.008
Ibolo = np.sqrt(Psat/Rbolo)
Vbolo = Ibolo*Rbolo
SI = -1/Vbolo

L0 = 10

NEP_Johnson = np.abs(1/(SI*L0))*np.sqrt(4*kb*Tbolo/Rbolo)

print('Ibolo        = {0:6.3e}'.format(Ibolo))
print('Vbolo        = {0:6.3e}'.format(Vbolo))
print('SI           = {0:6.3e}'.format(SI))
print('NEP_Johnson = {0:6.3e}'.format(NEP_Johnson))

Ibolo        = 3.536e-05
Vbolo        = 2.828e-07
SI           = -3.536e+06
NEP_Johnson = 9.398e-19
